# Fabric Analytics Roadshow - Deployment Notebook

This notebook deploys all lab assets into your Fabric workspace.

## Prerequisites
- Access to a Microsoft Fabric workspace
- Contributor or Admin permissions in the workspace
- Git repository cloned or uploaded to Fabric

## What This Notebook Does
1. Validates prerequisites and permissions
2. Creates Lakehouse and Warehouse artifacts
3. Deploys notebooks from the repository
4. Uploads Python scripts
5. Executes SQL setup scripts
6. Loads sample data
7. Validates deployment

In [ ]:
# Import required libraries
import json
import os
from pathlib import Path

print("✓ Libraries imported successfully")

## Step 1: Load Configuration

In [ ]:
# Load configuration from config.json
config_path = Path('config.json')

if config_path.exists():
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("✓ Configuration loaded successfully")
    print(f"  Workspace: {config['workspace']['name']}")
    print(f"  Lakehouse: {config['lakehouse']['name']}")
    print(f"  Warehouse: {config['warehouse']['name']}")
else:
    print("⚠ Configuration file not found. Using defaults.")
    config = {
        "workspace": {"name": "Fabric-Analytics-Roadshow"},
        "lakehouse": {"name": "RoadshowLakehouse"},
        "warehouse": {"name": "RoadshowWarehouse"}
    }

## Step 2: Validate Prerequisites

Check that we have the necessary permissions and environment.

In [ ]:
# TODO: Add validation logic
# - Check workspace access
# - Verify permissions
# - Validate Fabric environment

print("✓ Prerequisites validated")

## Step 3: Create Lakehouse

Create the Lakehouse artifact for data storage and processing.

In [ ]:
# TODO: Implement Lakehouse creation
lakehouse_name = config['lakehouse']['name']

print(f"Creating Lakehouse: {lakehouse_name}")
# Add Fabric API calls here

print(f"✓ Lakehouse '{lakehouse_name}' created successfully")

## Step 4: Create Warehouse

Create the Warehouse artifact for analytics workloads.

In [ ]:
# TODO: Implement Warehouse creation
warehouse_name = config['warehouse']['name']

print(f"Creating Warehouse: {warehouse_name}")
# Add Fabric API calls here

print(f"✓ Warehouse '{warehouse_name}' created successfully")

## Step 5: Deploy Notebooks

Upload all lab notebooks to the workspace.

In [ ]:
# TODO: Implement notebook deployment
notebooks_dir = Path('../notebooks')

if notebooks_dir.exists():
    notebook_files = list(notebooks_dir.rglob('*.ipynb'))
    print(f"Found {len(notebook_files)} notebooks to deploy")
    
    for notebook in notebook_files:
        print(f"  Deploying: {notebook.relative_to(notebooks_dir)}")
        # Add deployment logic here
    
    print("✓ All notebooks deployed successfully")
else:
    print("⚠ Notebooks directory not found")

## Step 6: Upload Python Scripts

Upload reusable Python modules to the workspace.

In [ ]:
# TODO: Implement Python script upload
scripts_dir = Path('../scripts')

if scripts_dir.exists():
    script_files = list(scripts_dir.glob('*.py'))
    print(f"Found {len(script_files)} Python scripts to upload")
    
    for script in script_files:
        print(f"  Uploading: {script.name}")
        # Add upload logic here
    
    print("✓ All scripts uploaded successfully")
else:
    print("⚠ Scripts directory not found")

## Step 7: Execute SQL Setup Scripts

Run SQL scripts to create warehouse objects.

In [ ]:
# TODO: Implement SQL script execution
sql_dir = Path('../sql/warehouse')

if sql_dir.exists():
    sql_files = list(sql_dir.glob('*.sql'))
    print(f"Found {len(sql_files)} SQL scripts to execute")
    
    for sql_file in sql_files:
        print(f"  Executing: {sql_file.name}")
        # Add execution logic here
    
    print("✓ All SQL scripts executed successfully")
else:
    print("⚠ SQL directory not found")

## Step 8: Load Sample Data

Load sample datasets for the lab exercises.

In [ ]:
# TODO: Implement sample data loading
data_dir = Path('../data/sample')

if data_dir.exists():
    print("Loading sample data...")
    # Add data loading logic here
    print("✓ Sample data loaded successfully")
else:
    print("⚠ Sample data directory not found")

## Step 9: Validate Deployment

Verify that all components were deployed correctly.

In [ ]:
# TODO: Implement deployment validation
print("Validating deployment...")

validation_results = {
    "lakehouse_created": False,
    "warehouse_created": False,
    "notebooks_deployed": False,
    "scripts_uploaded": False,
    "sql_executed": False,
    "data_loaded": False
}

# Add validation checks here

print("\n" + "="*50)
print("Deployment Summary:")
print("="*50)
for check, status in validation_results.items():
    status_icon = "✓" if status else "✗"
    print(f"{status_icon} {check.replace('_', ' ').title()}")

if all(validation_results.values()):
    print("\n✓ Deployment completed successfully!")
    print("\nYou can now proceed with the lab exercises.")
else:
    print("\n⚠ Some components failed validation. Please review the errors above.")